In [ ]:
#load dependencies
import tensorflow as tf
from tensorflow.keras import regularizers
import keras as k

import matplotlib.pyplot as plt
import torchsummary as summary
import numpy as np
np.random.seed(53702)
import pandas as pd
import os

!pip install shap --quiet
import shap
%matplotlib inline

!pip install scikit-survival --quiet

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.backend import dropout 

!pip install lifelines --quiet
import lifelines

from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
#for detailed description, see MrOs DNN surv decision of WL
def loss_lik_efron(y_true,y_pred):
  #loss function
  time = y_true[:,0]
  event = y_true[:,1]

  y_pred= k.backend.flatten(y_pred)
  y_pred=tf.cast(y_pred,tf.float32)

  n = tf.shape(time)[0]
  sort_index=tf.nn.top_k(time,k=n,sorted=True).indices

  time = k.backend.gather(reference=time,indices = sort_index)
  event = k.backend.gather(reference=event,indices = sort_index)  
  y_pred = k.backend.gather(reference=y_pred,indices = sort_index)  

  time_event = time * event
  unique_ftime = tf.unique(tf.boolean_mask(tensor = time_event, mask = tf.greater(time_event, 0))).y
  m = tf.shape(unique_ftime)[0]
  tie_count=tf.unique_with_counts(tf.boolean_mask(time_event, tf.greater(time_event, 0))).count
  ind_matrix = k.backend.expand_dims(time,0) - k.backend.expand_dims(time,1)
  ind_matrix = k.backend.equal(x=ind_matrix,y=k.backend.zeros_like(ind_matrix))
 

  time_count = k.backend.cumsum(tf.unique_with_counts(time).count)
  time_count = k.backend.cast(time_count - k.backend.ones_like(time_count), dtype = tf.int32)
  ind_matrix = k.backend.gather(ind_matrix, time_count)
  ind_matrix=tf.cast(ind_matrix,'float32')


  event=tf.cast(event,'float32')
  tie_haz = k.backend.exp(y_pred) * event
  tie_haz = k.backend.dot(ind_matrix, k.backend.expand_dims(tie_haz))
  event_index = tf.math.not_equal(tie_haz,0)
  tie_haz = tf.boolean_mask(tie_haz, event_index)

  tie_risk = y_pred * event
  tie_risk = k.backend.dot(ind_matrix, k.backend.expand_dims(tie_risk))
  tie_risk = tf.boolean_mask(tie_risk, event_index)

  cum_haz = k.backend.dot(ind_matrix, k.backend.expand_dims(k.backend.exp(y_pred)))
  cum_haz = k.backend.reverse(tf.cumsum(k.backend.reverse(cum_haz, axes = 1)), axes = 1)
  cum_haz = tf.boolean_mask(cum_haz, event_index)


  global likelihood

  if likelihood is None:
    likelihood = tf.Variable(0., trainable = False) 

  j = tf.cast(0, dtype = tf.int32)
  def loop_cond(j,a,b,c,d,e):
    return j < m

  def loop_body(j, tc, tr, th, ch, lik):
    l = tc[j]
    l = k.backend.cast(l, dtype = tf.float32)
    J = tf.linspace(start = tf.cast(0, tf.float32), stop = l-1, num = tf.cast(l, tf.int32))/l 
    Dm = ch[j] - J*th[j]
    lik = lik + tr[j] - tf.math.reduce_sum(tf.math.log(Dm))
    one = k.backend.ones_like(j)
    j_new = j + one
    return(list([j_new, tc, tr, th, ch, lik]))

  loop_out = tf.while_loop(cond = loop_cond, body = loop_body,
                            loop_vars = list([j, tie_count, tie_risk, tie_haz, cum_haz, likelihood]))
  log_lik = loop_out[-1]

  return(tf.negative(log_lik))
likelihood = None

#hyperparameters
L1s=[0.1,0.01,0.005,0.001,0.0005,0.0001]
Lrs=[1,0.1,0.01,0.005,0.001,0.0005,0.0001]
batch_sizes=[32,64,128,256,512,1024]
modelnumber=0

for zz in range(1,11):
  print("Iteration " + str(zz))
  StateToInv='EMG'
  WLIteration=zz
  os.chdir('/content/drive/MyDrive/Speciale/DL after 14-06/SHHS')
  print(os.listdir())
  index_train=pd.read_csv("indices_train.csv",header=None)
  index_test=pd.read_csv("indices_test.csv",header=None)

  cens_lab = pd.read_csv("SHHS_cens_lab.txt")

  cens_lab = cens_lab[["days", "isdead"]]

  os.chdir('/content/drive/MyDrive/Speciale/DL after 14-06/SHHS/' + StateToInv)
  features = pd.read_csv(StateToInv+"_WL_"+ str(WLIteration) +".txt")

  ###
  np.random.seed(53702)
  #randnums= np.random.randint(0,10000,len(cens_lab))/10000
  #features["RandomArray"]=randnums
  ### 
  print(np.shape(features))

  rows_with_nan = [index for index, row in features.iterrows() if row.isnull().any()]

  features=features.drop(axis=0, index=rows_with_nan)
  cens_lab=cens_lab.drop(axis=0,index=rows_with_nan)

  Rem_more = [index for index, row in cens_lab.iterrows() if row.isnull().any()]


  features=features.drop(axis=0, index=Rem_more)
  cens_lab=cens_lab.drop(axis=0,index=Rem_more)

  cens_lab=cens_lab.astype(int)
  ###
  npfeat=features.to_numpy()
  np_cens=cens_lab.to_numpy()
  nptrainidx=index_train.to_numpy().astype(int);
  nptestidx=index_test.to_numpy().astype(int);

  x_train_tmp=npfeat[nptrainidx,:]
  y_train_tmp=np_cens[nptrainidx,:]

  x_test_tmp=npfeat[nptestidx,:]
  y_test_tmp=np_cens[nptestidx,:]

  x_train=x_train_tmp.reshape(np.shape(x_train_tmp)[0],np.shape(x_train_tmp)[2])
  y_train=y_train_tmp.reshape(np.shape(y_train_tmp)[0],np.shape(y_train_tmp)[2])

  x_test=x_test_tmp.reshape(np.shape(x_test_tmp)[0],np.shape(x_test_tmp)[2])
  y_test=y_test_tmp.reshape(np.shape(y_test_tmp)[0],np.shape(y_test_tmp)[2])
  for ss in range(10):
    for q in L1s:
      for w in Lrs:
        for z in batch_sizes:
          print("Model:" + str(modelnumber) + "_BS:" + str(z) + "_LR:" + str(w) + "_L1:" + str(q))
          num_L1=q
          num_lr=w
          num_epoch=200
          batch_size=z

          model = Sequential()

          #model.build(input_shape=125)
          #model.add(Dense(1024, 
          #                activation='relu',
          #                kernel_regularizer=regularizers.L1(num_L1)))
          #model.add(Dropout(0.2))
          model.add(Dense(512, 
                          activation='relu',
                          kernel_regularizer=regularizers.L1(num_L1)))
          model.add(Dropout(0.2))

          model.add(Dense(128, 
                          activation='relu',
                          kernel_regularizer=regularizers.L1(num_L1)))
          model.add(Dropout(0.2))

          model.add(Dense(64, 
                    activation='relu',
                    kernel_regularizer=regularizers.L1(num_L1)))
          model.add(Dropout(0.2))

          model.add(Dense(34, 
                    activation='relu',
                    kernel_regularizer=regularizers.L1(num_L1)))
          model.add(Dropout(0.2))

          model.add(Dense(1, 
                          activation='relu'))
          
          callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

          model.compile(loss=loss_lik_efron,
                  optimizer=tf.keras.optimizers.Adam(num_lr),#tf.keras.optimizers.RMSprop(num_lr),#'adam',
                  metrics=None)
          
          history=model.fit(x=x_train, y=y_train, 
                      batch_size=batch_size, 
                      epochs=num_epoch, 
                      verbose=0, 
                      validation_split=0.1,
                      callbacks=[callback])
          
          predicts=model.predict(x_test)
          df = pd.DataFrame(y_test, columns=['T', 'E'])
          df["predicts"]=predicts
          if bool(abs(pd.isnull(df["predicts"][1])-1)):
            try:
              cph = lifelines.fitters.coxph_fitter.CoxPHFitter().fit(df, 'T', 'E')
              cindex=round(lifelines.utils.concordance_index(df['T'], -cph.predict_partial_hazard(df), df['E'])*100,1)

              print(cindex)

              plt.plot(history.history['loss'])
              plt.plot(history.history['val_loss'])

              modelnumber += 1
              modelname="Model:" + str(modelnumber) + "_BS:" + str(z) + "_LR:" + str(w) + "_L1:" + str(q) + "_CI:" + str(cindex)
              TitleName="Model loss DNNSurv - BS:" + str(z) + "_LR:" + str(w) + "_L1:" + str(q) + "_CI:" + str(cindex)
              plt.title(TitleName)
              plt.ylabel('loss')
              plt.xlabel('epoch')
              plt.legend(['train', 'test'], loc='upper left')
              plt.show()
              #plt.savefig(modelname+".png", bbox_inches='tight')
              #plt.clf()
            except Exception as e:
              print(e)